APARTADO 1)
Implementar la estructura de datos para el circuito. Elegir un circuito “solución” que será
el que supondremos que resulta dañado y hay que restaurar.

Para realizar este apartado asignaremos el valor de m y n que queremos y leeremos de un fichero txt la estructura del circuito,
el txt tendra una estrucutura:
TipoPuerta, entrada1, entrada2
Dónde TipoPuerta puede ser "OR", "AND", "XOR", "NAND", "NOT", "NOR" y "XNOR"
Dónde entrada es un número identificador de la entrada de la puerta en cuestión, este número debe estar contenido en el rango 
[-1,n*m+n-1]
-1 significa que no tiene entrada
[0,n-1] significa que la entrada es una de las entradas del circuito
[n,m*m+n-1] significa que la entrada es la salida de una puerta del circuito (importante definir el circuito de manera que cada puerta sólo pueda recibir como entradas la salida de las dos capas anteriores)


In [6]:
#Introducimos los parámetros m y n del circuito
m = 4
n = 3

#Leemos de un fichero la estructura del circuito, dicho fichero deberá contener tantas líneas como puertas se quieran introducir
# en el circuito, por cada puerta introduciremos una línea en el fichero de la siguiente manera:
# TipoPuerta, entrada1, entrada2
#TipoPuerta existente en {AND, OR, NAND, XOR, NOT}
# entrada1 y entrada2 existente en {-1, n*m+n-1}
circuitoSolucion = []
with open('Circuito1.txt', 'r') as fichero:
    id_puerta = n
    for línea in fichero:
        puerta, entrada1, entrada2 = tuple(línea.split(","))
        
        conexion = (int(entrada1), id_puerta)
        conexion2 = (int(entrada2), id_puerta)
        
        circuitoSolucion.append((puerta,id_puerta,[conexion,conexion2]))
        
        id_puerta += 1
print(circuitoSolucion)        


[('OR', 3, [(0, 3), (1, 3)]), ('NOT', 4, [(1, 4), (-1, 4)]), ('XNOR', 5, [(1, 5), (2, 5)]), ('NAND', 6, [(3, 6), (5, 6)]), ('OR', 7, [(3, 7), (4, 7)]), ('AND', 8, [(-1, 8), (-1, 8)]), ('XOR', 9, [(3, 9), (6, 9)]), ('NOR', 10, [(7, 10), (8, 10)]), ('AND', 11, [(7, 11), (5, 11)]), ('XNOR', 12, [(9, 12), (6, 12)]), ('NOT', 13, [(10, 13), (-1, 13)]), ('NOR', 14, [(7, 14), (11, 14)])]


APARTADO 2)
Implementar un mecanismo que actúe como simulador de circuitos (es decir, que dado un
vector de entrada con n bits calcule el vector de output del circuito), incluyendo una
opción de marcar ciertas conexiones y/o puertas como “defectuosas”.

In [7]:
def salida_Puerta(TipoPuerta, entrada1, entrada2):
    valorEntrada1 = False
    valorEntrada2 = False
    salida = False
    
    if(entrada1 == 1):
        valorEntrada1 = True
    if(entrada2 == 1):
        valorEntrada2 = True
    
    if(TipoPuerta == "OR"):
        salida = valorEntrada1 or valorEntrada2
    elif(TipoPuerta == "AND"):
        salida = valorEntrada1 and valorEntrada2
    elif(TipoPuerta == "NOT"):
        salida = not valorEntrada1
    elif(TipoPuerta == "NAND"):
        salida = not (valorEntrada1 and valorEntrada2)
    elif(TipoPuerta == "XOR"):
        salida = not valorEntrada1 == valorEntrada2
    elif(TipoPuerta == "NOR"):
        salida = not (valorEntrada1 or valorEntrada2)
    else:
        salida = valorEntrada1 == valorEntrada2
        
    return int(salida)


#Esta función se puede simplificar, pero es más legible si ponemos todos los casos cubiertos a la hora de explicar el funcionamiento
def salida_Circuito(Circuito, Entrada, Puertas_defectuosas, Conexiones_defectuosas):
    salidas_capas_intermedias = []
    salida = []
    for i in range(len(Circuito)):
        TipoPuerta = Circuito[i][0]
        id_Puerta = Circuito[i][1]
        conexiones_Puerta = Circuito[i][2]
        valorEntrada1 = 0
        valorEntrada2 = 0
        
        #Si la puerta está dañada la salida de la puerta va a ser 0
        if(id_Puerta in Puertas_defectuosas):
            #Las salidas de las capas intermedias las guardamos como una tupla(valorsalida, idpuerta) para identificar qué puertas han originado las salidas
            #Hay que diferenciar si la salida de la puerta es una salida intermedia o final del circuito
            if(id_Puerta < m*n):
                salidas_capas_intermedias.append((0,id_Puerta))
            else:
                salida.append(0)
            
        #Caso en el que la puerta no esté dañada
        else:
            
            #Si la conexión está dañada el valor para la entrada va a ser 0
            if(conexiones_Puerta[0] in Conexiones_defectuosas):
                valorEntrada1 = 0
            else:
                #Si no está dañada, calculamos el id de la entrada de la puerta, puede ser o bien una entrada del circuito o la salida de una puerta de capas anteriores
                id_entrada1 = conexiones_Puerta[0][0]
                
                 #Si el id de la entrada es igual a -1 significa que no tiene conexión dicha entrada, por tanto es como si tuviéramos un 0 como valor
                if(id_entrada1 == -1):
                    valorEntrada1 = 0
                
                #Si la entrada es una entrada del circuito
                elif(id_entrada1 <= n-1 and id_entrada1 > -1):
                    valorEntrada1 = Entrada[id_entrada1]
                    
                #Si la entrada es una salida de una puerta intermedia, el -n es porque en la lista salidas_capas_intermedias no tenemos las entradas del circuito por tanto el índice 0 se corresponde con el id 0+n
                else:
                    valorEntrada1 = salidas_capas_intermedias[id_entrada1 - n][0]
                    
                    
                    
                
            #Análogo al procedimiento anterior para la otra conexión de la puerta
            if(conexiones_Puerta[1] in Conexiones_defectuosas):
                valorEntrada2 = 0
            else:
                id_entrada2 = conexiones_Puerta[1][0]
                
                if(id_entrada2 == -1):
                    valorEntrada2 = 0
                
                elif(id_entrada2 <= n-1 and id_entrada2 > -1):
                    valorEntrada2 = Entrada[id_entrada2]
                    
                #Si la entrada es una salida de una puerta intermedia, el -n es porque en la lista salidas_capas_intermedias no tenemos las entradas del circuito por tanto el índice 0 se corresponde con el id 0+n
                else:
                    valorEntrada2 = salidas_capas_intermedias[id_entrada2 - n][0]
                
            #Finalmente una vez tenemos los valores de las entradas, calculamos la salida de la puerta con la función previamente definida salida_puerta
            #Hay que distinguir los casos en que la salida sea intermedia o final
            if(id_Puerta < m*n):
                salidas_capas_intermedias.append((salida_Puerta(TipoPuerta,valorEntrada1, valorEntrada2), id_Puerta))
            else:
                salida.append(salida_Puerta(TipoPuerta,valorEntrada1, valorEntrada2))

    return salida

Probamos el código anterior mediante un oráculo de tuplas entradas y salidas esperadas calculadas "a mano" para el circuito solución que estamos usando de ejemplo "Circuito1.txt"
1) Probaremos el Circuito sin añadir daños a puertas ni conexiones
Entrada 1,0,1 -> Salida esperada 0,1,0 
Entrada 0,0,0 -> Salida esperada 1,1,0
Entrada 1,1,1 -> Salida esperada 0,1,0


2) Probaremos el Circuito añadiendo daños a alguna puerta 
Entrada 1,0,1 con id's Puerta dañada 13 -> Salida esperada 0,0,0
Entrada 1,0,1 con id's Puertas dañadas 11,13 -> Salida esperada 0,0,0
Entrada 1,0,1 con id's Puertas dañadas 11,13,3 -> Salida esperada 1,0,0


3) Probaremos el Circuito añadiendo daños a alguna conexión
Entrada 1,0,1 con conexión dañada (7,10) -> Salida esperada 0,0,0
Entrada 1,0,1 con conexiones dañadas (7,10),(7,14) -> Salida esperada 0,0,1

4) Probaremos el Circuito añadiendo daños tanto en conexión como en puertas
Entrada 1,0,1 con conexión dañadas (7,10) y puertas dañadas 3 -> Salida esperada 1,0,0

In [8]:
print("Salidas sin daños")
print("------------------")
print(salida_Circuito(circuitoSolucion, [1,0,1],[],[]))
print(salida_Circuito(circuitoSolucion, [0,0,0],[],[]))
print(salida_Circuito(circuitoSolucion, [1,1,1],[],[]))



print("\nSalidas con daños en puertas")
print("------------------")
print(salida_Circuito(circuitoSolucion, [1,0,1],[13],[]))
print(salida_Circuito(circuitoSolucion, [1,0,1],[11,13],[]))
print(salida_Circuito(circuitoSolucion, [1,0,1],[11,13,3],[]))

print("\nSalidas con daños en conexiones")
print("------------------")
print(salida_Circuito(circuitoSolucion, [1,0,1],[],[(7,10)]))
print(salida_Circuito(circuitoSolucion, [1,0,1],[],[(7,10),(7,14)]))

print("\nSalidas con daños en conexiones y en puertas")
print("------------------")
print(salida_Circuito(circuitoSolucion, [1,0,1],[3],[(7,10)]))


Salidas sin daños
------------------
[0, 1, 0]
[1, 1, 0]
[0, 1, 0]

Salidas con daños en puertas
------------------
[0, 0, 0]
[0, 0, 0]
[1, 0, 0]

Salidas con daños en conexiones
------------------
[0, 0, 0]
[0, 0, 1]

Salidas con daños en conexiones y en puertas
------------------
[1, 0, 0]


APARTADO 3)
Construir una colección de pares de vectores de n bits (entrada,salida) que nos servirá de
referencia para representar el “comportamiento correcto” del circuito. Implementar un
método de auto-diagnóstico que compruebe si hay algún fallo respecto a dicha colección
de pares.


In [9]:
#Usado para obtener una lista de listas con todas las posibles combinaciones de n bits
import itertools

In [10]:
#Función que construye una lista de tuplas (entrada,salida) con todas las posibles combinaciones de n bits como entrada y las salidas 
#correspondientes del circuito sin tener ningún daño
def construir_pares_referencia():
    res = []
    entradas = list(map(list, itertools.product([0, 1], repeat=n)))
    for i in entradas:
        res.append((i, salida_Circuito(circuitoSolucion, i,[],[])))
    return res
 

#Función que dado un circuito y lista de puertas y conexiones dañadas calcula el número de de fallos producidos en las salidas con respecto a 
#las salidas de referencia del circuito solución sin tener ningún daño
def autodiagnostico(Circuito, Puertas_defectuosas, Conexiones_defectuosas):
    numFallos = 0
    for i in construir_pares_referencia():
        salida_circuito_defectuoso = salida_Circuito(Circuito, i[0],Puertas_defectuosas, Conexiones_defectuosas)
        for j in range(len(salida_circuito_defectuoso)):
            if(salida_circuito_defectuoso[j] != i[1][j] ):
                numFallos += 1
    return numFallos
        

In [11]:
print("Pares de referencia (entrada,salida) del comportamiento correcto del circuito:")
print("-----------------------------------------------------------------------------")
construir_pares_referencia()

Pares de referencia (entrada,salida) del comportamiento correcto del circuito:
-----------------------------------------------------------------------------


[([0, 0, 0], [1, 1, 0]),
 ([0, 0, 1], [1, 1, 0]),
 ([0, 1, 0], [0, 1, 0]),
 ([0, 1, 1], [0, 1, 0]),
 ([1, 0, 0], [0, 1, 0]),
 ([1, 0, 1], [0, 1, 0]),
 ([1, 1, 0], [0, 1, 0]),
 ([1, 1, 1], [0, 1, 0])]

In [12]:
print("Nº de fallos en las salidas del circuitoSolución si se daña las puertas con id 4 y 11, y las conexiones (3,6), (6,12)")
print("------------------------------------------------------------------------------------------------")
print("Nº fallos =",autodiagnostico(circuitoSolucion, [4,11],[(3,6),(6,12)]))

Nº de fallos en las salidas del circuitoSolución si se daña las puertas con id 4 y 11, y las conexiones (3,6), (6,12)
------------------------------------------------------------------------------------------------
Nº fallos = 12


APARTADO 4)
Implementar un algoritmo genético que dado un circuito dañado, busque “buenos”
circuitos alternativos, cambiando la programación de las puertas del circuito y/o las
conexiones entre ellas. El algoritmo genético debe trabajar sin conocer qué puertas son las
dañadas o defectuosas.

In [13]:
#Introducir los daños en el circuito
#Los daños en puertas se introducen mediante una lista de números identificadores, cada número identifica una entrada o una puerta del circuito,
    #de [0, n-1] identifica las entradas del circuito de [n,n*m+n-1] identifica las puertas
    
puertas_defectuosas_alg = [4,11]

#Los daños en conexiones se introducen mediante una lista de tuplas (entrada,salida):

conexiones_defectuosas_alg = [(3,6),(6,12)]


In [14]:
import random
from deap import base, creator, tools, algorithms
import numpy

In [15]:
creator.create('Fitness', base.Fitness, weights=(-1.0,))

C:\Users\Javier\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [16]:
creator.create('Individuo', list, fitness = creator.Fitness)

C:\Users\Javier\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individuo' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [17]:
caja_de_herramientas = base.Toolbox()

In [18]:


#Función de generación de genes para el cromosoma, debemos distinguir que para cada puerta tenemos 3 genes, el primero indica
# el tipo de puerta y los 2 genes siguientes las entradas.
# Para hacer el algoritmo más eficiente, no daremos la posibilidad de que se generen cromosomas erróneos, en el sentido de que exista
# la posibilidad de que una puerta tenga como entrada la salida de otra puerta de una capa posterior.
#Aún existe la posibilidad de que se generen cromosomas erróneos en el sentido de que la entrada de una puerta provenga de un
# elemento(entrada circuito o puerta) no perteneciente  a las 2 capas anteriores, esta situación se penalizará en el fitness.
#Corregido: se daba la posibilidad de que existieran conexiones entre puertas de la misma capa, se introduce 3*n en el divisor y el 
#resultado de ese cociente se multiplica por n

def contador_llamadas(func):
     func.count += 1

def generador_genes():
    #Reseteamos el contador de llamadas cuando ya hemos finalizado de generar los genes de un cromosoma entero
    if(generador_genes.count == 3*n*m):
        generador_genes.count = 0
    
    valor = 0
    if(generador_genes.count % 3 == 0):
        valor = random.randint(0,6)
    else:
        valor = random.randint(-1,n-1+n*(generador_genes.count//(3*n)))
    
    contador_llamadas(generador_genes)
    
    return valor

generador_genes.count = 0
        

caja_de_herramientas.register('gen', generador_genes)

In [19]:
caja_de_herramientas.register('individuo', tools.initRepeat,
                              container=creator.Individuo, func=caja_de_herramientas.gen, n=3*n*m)

In [20]:
caja_de_herramientas.register('población', tools.initRepeat,
                              container=list, func=caja_de_herramientas.individuo, n=2000)

In [21]:
def fenotipo(individuo):
    circuito = []
    id_puerta_actual = n
    for i in range(0,len(individuo),3):
        conexion = (individuo[i+1], id_puerta_actual)
        conexion2 = (individuo[i+2],id_puerta_actual)
        
        if(individuo[i] == 0):
            circuito.append(("OR",id_puerta_actual, [conexion,conexion2]))
        elif(individuo[i] == 1):
            circuito.append(("AND",id_puerta_actual, [conexion,conexion2]))
        elif(individuo[i] == 2):
            circuito.append(("XOR",id_puerta_actual, [conexion,conexion2]))
        elif(individuo[i] == 3):
            circuito.append(("NAND",id_puerta_actual, [conexion,conexion2]))
        elif(individuo[i] == 4):
            circuito.append(("NOR",id_puerta_actual, [conexion,conexion2]))
        elif(individuo[i] == 5):
            circuito.append(("XNOR",id_puerta_actual, [conexion,conexion2]))
        else:
            circuito.append(("NOT",id_puerta_actual, [conexion,conexion2]))
        
        id_puerta_actual += 1
            
    return circuito

def evaluar_individuo(individuo):
    circuito = fenotipo(individuo)
    numFallosSalidas = autodiagnostico(circuito, puertas_defectuosas_alg, conexiones_defectuosas_alg)
    numErroresEntradas = 0
    
    for i in circuito:
        conexion1 = i[2][0]
        conexion2 = i[2][1]
        if(conexion1[0] < (conexion1[1]-2*n-conexion1[1]%n) and conexion1[0] != -1):
            numErroresEntradas += 1
        if(conexion2[0] < (conexion2[1]-2*n-conexion2[1]%n) and conexion2[0] != -1):
            numErroresEntradas += 1
        
    #El coeficiente 2**n*n+1 es para verificar que toda solución errónea tenga peor fitness que la peor solución correcta posible
    return (numFallosSalidas + numErroresEntradas * (2**n*n+1),)

caja_de_herramientas.register('evaluate', evaluar_individuo)

In [22]:
#Definimos un test para comprobar la creación de un individuo/cromosoma, la representación del fenotipo para el mismo y la evaluación de su fitness
random.seed(12345)
cromosoma = caja_de_herramientas.individuo()
print("Cromosoma:")
print (cromosoma)
print("\nFenotipo del cromosoma:")
print(fenotipo(cromosoma))

print("\nEvaluación del cromosoma:")
print(evaluar_individuo(cromosoma))


#Podemos observar que el cromosoma y su fenotipo es correcto, excepto por lo comentado anteriormente, se pueden crear conexiones
# con elementos previos a las dos capas anteriores, lo penalizaremos en el fitness


Cromosoma:
[3, -1, 1, 6, 1, 0, 2, 2, 0, 2, -1, 5, 3, 1, 3, 5, 0, 3, 4, 1, 4, 5, 0, 7, 5, 5, 8, 4, 1, 1, 1, 10, 0, 1, 4, 4]

Fenotipo del cromosoma:
[('NAND', 3, [(-1, 3), (1, 3)]), ('NOT', 4, [(1, 4), (0, 4)]), ('XOR', 5, [(2, 5), (0, 5)]), ('XOR', 6, [(-1, 6), (5, 6)]), ('NAND', 7, [(1, 7), (3, 7)]), ('XNOR', 8, [(0, 8), (3, 8)]), ('NOR', 9, [(1, 9), (4, 9)]), ('XNOR', 10, [(0, 10), (7, 10)]), ('XNOR', 11, [(5, 11), (8, 11)]), ('NOR', 12, [(1, 12), (1, 12)]), ('AND', 13, [(10, 13), (0, 13)]), ('AND', 14, [(4, 14), (4, 14)])]

Evaluación del cromosoma:
(183,)


In [23]:
caja_de_herramientas.register('mate', tools.cxUniform, indpb=0.1)
#Tal y como hemos definido la estructura de nuestros cromosomas (diferentes significados según la posición del gen), 
#tenemos una única opción de mutación muy específica,
#la mutación uniforme, indicando rangos diferentes según la posición del gen en el cromosoma
listLow = []
listUp = []
for i in range(3*n*m):
    if(i%3==0):
        listLow.append(0)
        listUp.append(6)
    else:
        listLow.append(-1)
        listUp.append(n-1+n*(i//(3*n)))
        
caja_de_herramientas.register('mutate', tools.mutUniformInt,low=listLow, up= listUp, indpb=0.1)

In [24]:
caja_de_herramientas.register('select', tools.selTournament, tournsize=3)

In [25]:
# Estadísticas sobre el fitness de los individuos: mínimo, media y máximo
estadísticas = tools.Statistics(lambda ind: ind.fitness.values)
estadísticas.register("mínimo", numpy.min)
estadísticas.register("media", numpy.mean)
estadísticas.register("máximo", numpy.max)

# Salón de la fama para recopilar los tres mejores individuos de todas las generaciones
salón_fama = tools.HallOfFame(1)

In [26]:
random.seed(12345)
población_inicial = caja_de_herramientas.población()
print("Imprimimos las estadísticas del proceso del algoritmo:")
print("------------------------------------------------------")
población_final, registro = algorithms.eaSimple(población_inicial,
                                                caja_de_herramientas,
                                                cxpb=0.5,  # Probabilidad de cruzamiento
                                                mutpb=0.3,  # Probabilidad de mutación
                                                ngen=20,  # Número de generaciones
                                                stats=estadísticas,
                                                halloffame=salón_fama)



Imprimimos las estadísticas del proceso del algoritmo:
------------------------------------------------------
gen	nevals	mínimo	media 	máximo
0  	2000  	8     	126.76	266   
1  	1308  	6     	92.826	208   
2  	1265  	4     	68.477	187   
3  	1334  	4     	50.603	139   
4  	1283  	2     	35.6755	133   
5  	1226  	2     	24.3825	118   
6  	1262  	2     	16.3925	114   
7  	1286  	0     	12.7815	91    
8  	1330  	0     	10.9595	87    
9  	1310  	0     	9.438  	139   
10 	1339  	0     	8.7085 	89    
11 	1258  	0     	8.1535 	85    
12 	1302  	0     	7.496  	85    
13 	1321  	0     	7.5755 	108   
14 	1345  	0     	7.616  	93    
15 	1253  	0     	6.657  	104   
16 	1281  	0     	6.311  	91    
17 	1336  	0     	6.9485 	88    
18 	1324  	0     	6.067  	85    
19 	1312  	0     	6.5595 	87    
20 	1308  	0     	6.617  	107   


In [70]:
print('La mejor solución de reconfiguración encontrada es:\n')
for individuo in salón_fama:
    print('Puertas={1}; Fitness: {0}\n'.format(
        individuo.fitness.values[0], fenotipo(individuo)))
    print("\nDiagnóstico:")
    print("------------")
    if(individuo.fitness.values[0] == 0):
        print("¡Se ha logrado una auto-reparación completa del circuito, rendimiento del 100%!")
    else:
        print("No se ha logrado una auto-reparación completa del circuito, el rendimiento es de: {0}%".format(
            (2**n*n-individuo.fitness.values[0])/(2**n*n)*100))


La mejor solución de reconfiguración encontrada es:

Puertas=[('XNOR', 3, [(2, 3), (0, 3)]), ('OR', 4, [(-1, 4), (0, 4)]), ('AND', 5, [(2, 5), (-1, 5)]), ('NOT', 6, [(1, 6), (0, 6)]), ('AND', 7, [(0, 7), (1, 7)]), ('OR', 8, [(1, 8), (0, 8)]), ('NOT', 9, [(5, 9), (6, 9)]), ('XOR', 10, [(4, 10), (7, 10)]), ('OR', 11, [(-1, 11), (7, 11)]), ('XNOR', 12, [(6, 12), (8, 12)]), ('NAND', 13, [(6, 13), (11, 13)]), ('XOR', 14, [(7, 14), (10, 14)])]; Fitness: 0.0


Diagnóstico:
------------
¡Se ha logrado una auto-reparación completa del circuito, rendimiento del 100%!
